**Utilizing External Data in Our Pre-processing Pipeline**

Based on the recent discussion in the [Trustii Team forum](https://app.trustii.io/datasets/1519/forums/125/messages), it's been confirmed that we have the flexibility to incorporate any dataset into our pre-processing pipeline, with the caveat that this dataset should not contain any labels from our test set.

In line with this, I've acquired an additional dataset titled **"Sirene: Fichier StockEtablissement du 26 Mars 2024"**. This comprehensive dataset can be manually downloaded from [Data.gouv.fr](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/#/resources), where you can also find detailed information about the file contents and column descriptions. Alternatively, you can directly initiate the download [here](https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576).

This extensive dataset encompasses a wide array of entities in France, including essential information like Siret and Siren numbers, among other data. We've subsequently merged this with our dataset for enhanced analysis.

**Please Note:** The size of the CSV file is substantial, and it cannot be read in its entirety at once. Below, I've provided a script that reads the data in chunks and extracts only the rows matching the Siret numbers in our dataset.

### Comprehensive Guide to Utilize the Dataset:

1. **Download the Dataset:**
   Download the zip file "Sirene: Fichier StockEtablissement du 26 Mars 2024" from the [provided link](https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576).

2. **Unzip the File:**
   After downloading, extract the contents of the zip file.

3. **Set Up the File Path:**
   In the script below, assign `file_path` to the location of the unzipped file.



# Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
pd.set_option('display.max_columns', None)

# Data Loading and Preparation

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = test[['trustii_id']]
test.drop(['trustii_id'], axis=1, inplace=True)

In [3]:
train['train'] = 1
test['train'] = 0
train_test = pd.concat([train, test])

In [4]:
train_test['SIRET'] = train_test['SIRET'].apply(lambda x: float(x) if x.isnumeric() else 0)
siret_list = train_test['SIRET'].values

# SIRET Data Processing 

In [5]:
# File path
file_path = r'Path/to/uncompressed/file'

# Initialize an empty DataFrame to store the filtered rows
filtered_df = pd.DataFrame()

# Process the file in chunks
chunksize = 100000  # You can adjust the chunk size based on your memory capacity
for chunk in tqdm(pd.read_csv(file_path, chunksize=chunksize)):
    filtered_chunk = chunk[chunk['siret'].isin(siret_list)]
    filtered_df = pd.concat([filtered_df, filtered_chunk])

In [6]:
# Calculate the percentage of NaN values in each column
percent_nan = filtered_df.isnull().mean() * 100

# Identify columns where the percentage of NaN values is more than 50%
columns_to_drop_nan = percent_nan[percent_nan > 50].index

# Identify columns with only one unique value
columns_to_drop_unique = filtered_df.columns[filtered_df.nunique() <= 1]

# Combine the two sets of columns to drop
columns_to_drop = columns_to_drop_nan.union(columns_to_drop_unique)

# Drop these columns from the DataFrame
df = filtered_df.drop(columns=columns_to_drop)


In [7]:
print("the columns we are left with are ", list(df.columns))

the columns we are left with are  ['siren', 'nic', 'siret', 'statutDiffusionEtablissement', 'dateCreationEtablissement', 'trancheEffectifsEtablissement', 'etablissementSiege', 'nombrePeriodesEtablissement', 'numeroVoieEtablissement', 'typeVoieEtablissement', 'libelleVoieEtablissement', 'codePostalEtablissement', 'libelleCommuneEtablissement', 'codeCommuneEtablissement', 'dateDebut', 'etatAdministratifEtablissement', 'activitePrincipaleEtablissement', 'nomenclatureActivitePrincipaleEtablissement', 'caractereEmployeurEtablissement']


In [8]:
df['dateCreationEtablissement'] = pd.to_datetime(df['dateCreationEtablissement'])

# Create new columns in the DataFrame
df['day_variable'] = df['dateCreationEtablissement'].dt.day
df['month_variable'] = df['dateCreationEtablissement'].dt.month
df['year_variable'] = df['dateCreationEtablissement'].dt.year
df = df.drop(['dateCreationEtablissement'], axis=1)

# Splitting the 'activitePrincipaleEtablissement' into multiple columns
df['section'] = df['activitePrincipaleEtablissement'].str[0]
df['division'] = df['activitePrincipaleEtablissement'].str[1:3]
df['group'] = df['activitePrincipaleEtablissement'].str[3]
df['class'] = df['activitePrincipaleEtablissement'].str[4]
df['subclass'] = df['activitePrincipaleEtablissement'].str[4:]

df['trancheEffectifsEtablissement'] = df['trancheEffectifsEtablissement'].fillna(-1).apply(lambda x:  float(x) if x!='NN' else 0)
df['numeroVoieEtablissement'].fillna(-1).apply(lambda x:  float(x) if x!='[ND]' else 0)
df = df.drop(['libelleCommuneEtablissement', 'codePostalEtablissement'], axis=1)

In [9]:
train_test = pd.merge(train_test, df, left_on='SIRET', right_on='siret', how='left')

In [10]:
train_test.isna().mean()

APP_Libelle_etablissement                      0.000031
SIRET                                          0.000000
Adresse_2_UA                                   0.014295
Code_postal                                    0.000000
Libelle_commune                                0.000000
Numero_inspection                              0.000000
Date_inspection                                0.000000
APP_Libelle_activite_etablissement             0.000000
Synthese_eval_sanit                            0.300019
Agrement                                       0.743732
geores                                         0.026677
filtre                                         0.257441
ods_type_activite                              0.000000
train                                          0.000000
siren                                          0.003427
nic                                            0.003427
siret                                          0.003427
statutDiffusionEtablissement                   0

# Feature Engineering

Create new features:

- `paris`: Flag indicating if the establishment is in Paris.
- `type_adresse`: Categorize address type based on keywords.
- `Agrement`: Flag indicating missing 'Agrement' value.
- `filtre`: Flag indicating missing 'filtre' value.
- `APP_Libelle_etablissement_count`: Count occurrences of 'APP_Libelle_etablissement'.


In [11]:
train_test['paris'] = train_test.Libelle_commune.apply(lambda x: int('paris' in x.lower()) )
train_test['type_adresse'] = train_test["Adresse_2_UA"].astype(str).apply(lambda x: 'rue' if 'rue' in x.lower() else ('av' if 'av' in x.lower() else ('ecole' if 'ecole' in x.lower() else 'autre' )))
train_test['Agrement'] = train_test['Agrement'].apply(lambda x: 1 if pd.isna(x) else 0)

# "APP_Libelle_activite_etablissement" and  'filtre' are the same
train_test['filtre'] = train_test['filtre'].apply(lambda x: 1 if pd.isna(x) else 0)

count = train_test['APP_Libelle_etablissement'].value_counts()
train_test['APP_Libelle_etablissement_count'] = train_test['APP_Libelle_etablissement'].map(count)

# Activities 

The following code first processes the "APP_Libelle_activite_etablissement" column by standardizing the activity descriptions and splitting them into individual activities. Then, it creates new columns for various food-related categories and assigns a value of 1 to each category if the establishment's activities include any activity from that category's predefined list.

In [12]:
# Function to standardize and split activities
def process_activities(activity):
    if pd.isna(activity) or activity == '_':
        return []
    # Standardize: lower case and replace separators
    standardized = activity.lower().replace('-', '|').replace('/', '|')
    # Split into list of activities
    return set(standardized.split('|'))

# Apply the function to the column
train_test['activities'] = train_test['APP_Libelle_activite_etablissement'].apply(process_activities)

activity_categories = {
    "Meat and Poultry Processing": [
        'boucherie',
        "chaîne d'abattage d'animaux boucherie",
        "chaîne d'abattage de volaille",
        'charcuterie',
        'découpe de viande de boucherie',
        'découpe de viandes de volailles',
        'entreposage de viande de boucherie',
        'entreposage de viande de volailles',
        'rayon boucherie',
        'vente en gros de viande de boucherie',
        'vh_ vsm_ préparation viandes boucherie',
        'vh_ vsm_ préparation viandes volailles lagomorphes',
        "production d'abats",
        'traitement du gibier sauvage',
        'transformation de produits carnés',
        'gibier ongulé élevage',
        'lagomorphe',
        'petit gibier',
        'découpe de gibier sauvage',
        "découpe de gros gibier d'élevage",
    ],
    "Dairy and Egg Products": [
        'collecte de lait',
        "centre d'emballage des oeufs",
        "collecteur d'oeufs",
        'découpe de fromages',
        'entreposage de produits laitiers',
        'rayon fromagerie',
        "production d'œuf liquide d'ovoproduit et produits à base œuf",
        'transformation de lait ou produits laitiers',
        'fromagerie',
        'déshydratation de lait ou produits laitiers',
        'elevage de bovins',
    ],
    "Fish and Seafood Processing": [
        "abattage de produits de l'aquaculture",
        'entreposage de produits de la pêche',
        'expédition de coquillages',
        'mareyage et préparation de produits de la pêche',
        'poissonnerie',
        'rayon poissonnerie',
        'vente en gros produits de la pêche',
        "plateforme d'éclatement de produits de la pêche",
        'transformation de produits de la pêche',
        'vivier produits de la pêche',
    ],
    "Bakery and Confectionery": [
        'boulangerie',
        'pâtisserie',
        'chocolatier',
        'viennoiserie',
        'glacier',
    ],
    "General Food Retail and Services": [
        'alimentation générale',
        'commerce alimentaire',
        'distribution automatique',
        'libre service',
        'producteur fermier',
        'restaurant',
        'restauration collective',
        'responsable de restauration collective',
        'rayon traiteur',
        'traiteur',
    ],
    "Specialized Food Processing and Handling": [
        "collecte de gibier sauvage",
        "production de produits à base d'escargots",
        'production de produits à base de grenouilles',
        'préparation de produits composés',
        'purification',
        "caves d'affinage",
        'halle à m.',
        'infrastructure de marché',
        'entreposage non spécialisé de denrées alimentaires',
        'entreposage de distribution de denrées alimentaires',
        'transport de denrées alimentaires',
        'pêche de production primaire',
        'gestion administrative',
        "salle d'abattage à la farme",
        "production de collagène, de gélatine ou de phr",
        'autres activités de remise directe',
        'primeur',
        'métier de bouche',
    ]
}

for category, activities in activity_categories.items():
    train_test[category] = train_test['activities'].apply(lambda x: 1 if any(activity in x for activity in activities) else 0)
train_test['activities'] = train_test['activities'].apply(len)

## Numero Inspection

In [13]:
train_test['numero_inspection_1'] = train_test['Numero_inspection'].apply(lambda x: x[:2]).astype(float)
train_test['numero_inspection_2'] = train_test['Numero_inspection'].apply(lambda x: int(x[3:-2]))

## Postal code

In [14]:
train_test['region_postal_code'] = train_test['Code_postal'].apply(lambda x: x[:2] if x[:2]!='AD' and x[:2]!='0[' else 100 ).astype(float)
train_test['sub_region_postal_code'] = train_test['Code_postal'].apply(lambda x: x[2:] if x[2:]!= '0NR' and x[2:]!='ND]' else 0).astype(float)

count = train_test['region_postal_code'].value_counts()
train_test['region_postal_code_count'] = train_test['region_postal_code'].map(count)
train_test['Code_postal'] = pd.to_numeric(train_test['Code_postal'], errors='coerce')

## Siret count

In [15]:
count = train_test['SIRET'].value_counts()
train_test['SIRET_count'] = train_test['SIRET'].map(count)

## Date Inspection

In [16]:
# Ensure 'Date_inspection' is in datetime format
train_test['Date_inspection'] = pd.to_datetime(train_test['Date_inspection'])

# Extract year, month, and day directly from datetime.datetime objects
train_test['month'] = train_test['Date_inspection'].apply(lambda x: x.month)
train_test['day'] = train_test['Date_inspection'].apply(lambda x: x.day)

first_inspection_date = train_test['Date_inspection'].min()

train_test['day_of_week'] = train_test['Date_inspection'].apply(lambda x: x.weekday())

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

train_test['season'] = train_test['Date_inspection'].apply(lambda x: get_season(x.month))

train_test.drop(['Date_inspection'], axis=1, inplace=True)

## Geolocation Processing

- `Extract_geo`: Extract latitude and longitude from 'geores'.
- `Impute_geo`: Impute missing values with mean.
- `Distance_fixed_point`: Calculate distance to a fixed point in France.
- `Cluster_geo`: Cluster establishments based on latitude and longitude.
- `Region_center_coordinates`: Calculate region center coordinates.


In [17]:
train_test[['lat', 'lon']] = train_test['geores'].str.split('_', expand=True)

# Convert 'lat' and 'lon' from string to float
train_test['lat'] = train_test['lat'].astype(float)
train_test['lon'] = train_test['lon'].astype(float)

train_test['lat'] = train_test['lat'].fillna(train_test['lat'].mean())
train_test['lon'] = train_test['lon'].fillna(train_test['lon'].mean())
train_test.drop(['geores'], axis=1, inplace=True)

In [18]:
# ! maybe consider the distance from the centre of each city
from math import cos, sin, radians, sqrt

# 2. Haversine Distance
def haversine(lat_lon1, lat_lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = map(np.radians, lat_lon1)
    lat2, lon2 = map(np.radians, lat_lon2)
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

fixed_point = (46.824, 2.298)  # roughly the centre of France (latitude, longitude)
train_test['distance_to_fixed_point'] = train_test.apply(lambda row: haversine((row['lat'], row['lon']), fixed_point), axis=1)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(train_test[['lat', 'lon']])

# Getting the cluster assignments
labels = kmeans.labels_

train_test['cluster'] = labels

lat_center = train_test.groupby('region_postal_code').lat.mean()
lon_center = train_test.groupby('region_postal_code').lon.mean()
train_test['region_lat_center'] = train_test['region_postal_code'].map(lat_center)
train_test['region_lon_center'] = train_test['region_postal_code'].map(lon_center)

# Label Encoding

In [19]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Loop through each object type column and apply LabelEncoder
for col in train_test.columns:
    if train_test[col].dtype == 'object' and col!= 'Synthese_eval_sanit':
        train_test[col] = le.fit_transform(train_test[col])


# Target Encoding
Encode 'Synthese_eval_sanit' with appropriate mapping

In [20]:
train_test.Synthese_eval_sanit.unique()
encoding = {'Satisfaisant':2, 'Très satisfaisant':3, 'A améliorer':1, 'A corriger de manière urgente':0, np.nan:np.nan}
train_test['Synthese_eval_sanit'] = train_test['Synthese_eval_sanit'].map(encoding)

# Split Data

In [21]:
train_test = train_test.fillna(train_test.mean())
train = train_test[train_test.train == 1].drop(['train'], axis=1)

test = train_test[train_test.train == 0].drop(['train', 'Synthese_eval_sanit'], axis=1)

In [22]:
X = train.drop(["Synthese_eval_sanit"], axis=1)
y = train.Synthese_eval_sanit

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Model Training and Evaluation

In [23]:
from sklearn.metrics import accuracy_score

def accuracy(pred_probs, dmatrix):
    labels = dmatrix.get_label()
    acc = accuracy_score(labels, pred_probs)
    return '1-merror', acc

In [24]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix
params = {
 'n_jobs':-1,
 'disable_default_eval_metric':1,
 'num_class':4,
 'objective':'multi:softmax',
 'random_state':0,
 'colsample_bylevel': 0.6,
 'colsample_bynode': 0.7,
 'colsample_bytree': 0.9,
 'device': 'cpu',
 'learning_rate': 0.015,
 'max_depth': 8,
 'n_estimators': 2000,
 'num_parallel_tree': 2,
 'random_state': 0,
 'reg_lambda': 1.3,
 'tree_method': 'gpu_hist'}

clf = xgb.XGBClassifier(**params)

# Fit the classifier to the training data
clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=accuracy, verbose=100)

[0]	validation_0-1-merror:0.67385	validation_1-1-merror:0.63109
[100]	validation_0-1-merror:0.73329	validation_1-1-merror:0.67519
[200]	validation_0-1-merror:0.76334	validation_1-1-merror:0.68644
[300]	validation_0-1-merror:0.79422	validation_1-1-merror:0.69217
[400]	validation_0-1-merror:0.82631	validation_1-1-merror:0.69349
[500]	validation_0-1-merror:0.85289	validation_1-1-merror:0.69746
[600]	validation_0-1-merror:0.87654	validation_1-1-merror:0.69857
[700]	validation_0-1-merror:0.89694	validation_1-1-merror:0.70055
[800]	validation_0-1-merror:0.91597	validation_1-1-merror:0.70143
[900]	validation_0-1-merror:0.93190	validation_1-1-merror:0.70320
[1000]	validation_0-1-merror:0.94492	validation_1-1-merror:0.70187
[1100]	validation_0-1-merror:0.95694	validation_1-1-merror:0.70165
[1200]	validation_0-1-merror:0.96482	validation_1-1-merror:0.70408
[1300]	validation_0-1-merror:0.97326	validation_1-1-merror:0.70364
[1400]	validation_0-1-merror:0.97905	validation_1-1-merror:0.70540
[1500]	

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=0.7, colsample_bytree=0.9,
              device=None, disable_default_eval_metric=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1800, n_jobs=None, num_class=4, ...)

In [25]:
# Predict on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

Accuracy:  0.7034178610804851
Confusion Matrix: 
 [[   0    0   21    1]
 [   0   25  195   45]
 [   0    3 2230  369]
 [   0    3  708  935]]


In [26]:
# train on the whole dataset
clf = xgb.XGBClassifier(**params)
clf.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=0.7, colsample_bytree=0.9,
              device=None, disable_default_eval_metric=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1800, n_jobs=None, num_class=4, ...)

In [27]:
reverse_encoding = {v:k for k,v in encoding.items()}

# Submission file

In [28]:
submission['Synthese_eval_sanit'] = clf.predict(test)
submission['Synthese_eval_sanit'] = submission['Synthese_eval_sanit'].map(reverse_encoding)

In [29]:
submission.to_csv('sub.csv', index=False)